## 846. Hand of Straights
- Description:
  <blockquote>
    Alice has some number of cards and she wants to rearrange the cards into groups so that each group is of size `groupSize`, and consists of `groupSize` consecutive cards.

    Given an integer array `hand` where `hand[i]` is the value written on the `i<sup>th</sup>` card and an integer `groupSize`, return `true` if she can rearrange the cards, or `false` otherwise.

    **Example 1:**

    ```
    Input: hand = [1,2,3,6,2,3,4,7,8], groupSize = 3
    Output: true
    Explanation: Alice's hand can be rearranged as [1,2,3],[2,3,4],[6,7,8]

    ```

    **Example 2:**

    ```
    Input: hand = [1,2,3,4,5], groupSize = 4
    Output: false
    Explanation: Alice's hand can not be rearranged into groups of 4.


    ```

    **Constraints:**

    -   `1 <= hand.length <= 10<sup>4</sup>`
    -   `0 <= hand[i] <= 10<sup>9</sup>`
    -   `1 <= groupSize <= hand.length`
  </blockquote>

- URL: [Problem_URL](https://leetcode.com/problems/hand-of-straights/description/)

- Topics: Problem_topic

- Difficulty: Medium

- Resources: example_resource_URL

### Solution 1, Freq Map + Heap sol

Let n be the size of the hand array and k be groupSize.
- Time Complexity: O(n + k log k) = O(n log n)


Initial checks: O(1)

Building Counter: O(n)

Heap construction: O(k) to build list + O(k) to heapify → O(k)

Main loop:
    The outer while min_heap runs once per group, and there are n / groupSize groups.
    But more precisely, each card is processed exactly once across all groups.
    For each group, we iterate groupSize times → total iterations = n
    Inside the inner loop:
        Accessing/updating card_count → O(1)
        When a count hits zero, we do heapq.heappop(min_heap) → O(log k)

So, each card removal from heap happens once per unique card, and there are at most k such pops.

Therefore, total cost of all heappop operations: O(k log k)

The rest of the inner loop (checking and decrementing counts) is O(1) per card → total O(n)

Overall time:
    O(n) for counting
    O(k) for heapify
    O(n) for processing all cards
    O(k log k) for heap pops

Since k ≤ n, we can write:

Time Complexity = O(n + k log k) = O(n log n) in the worst case
(e.g., when all cards are distinct → k = n)

But if there are many duplicates (k << n), it’s closer to O(n + k log k).

However, in standard Big-O analysis (worst-case), we assume k = O(n)

- Space Complexity: O(N)
card_count = Counter(hand)        # O(n)
min_heap = list(card_count.keys())  # O(k), where k = number of unique cards
heapq.heapify(min_heap)           # in-place, no extra space

Apply Code

    n = len(hand)
    k = number of distinct card values → at most n

So, space complexity = O(n).

In [ ]:
class Solution:
    def isNStraightHand(self, hand: List[int], groupSize: int) -> bool:
        hand_size = len(hand)

        if hand_size % groupSize != 0:
            return False

        # Counter to store the count of each card value
        card_count = Counter(hand)

        # Min-heap to process the cards in sorted order
        min_heap = list(card_count.keys())
        heapq.heapify(min_heap)

        # Process the cards until the heap is empty
        while min_heap:
            current_card = min_heap[0]  # Get the smallest card value
            # Check each consecutive sequence of groupSize cards
            for i in range(groupSize):
                if card_count[current_card + i] == 0:
                    return False
                card_count[current_card + i] -= 1
                
                # The only time you should remove a card from the min-heap is when its count hits zero and it is the smallest remaining card (i.e., the one at the top of the heap). If not, the hand cannot be partitioned into valid straight groups.
                # If a card becomes unavailable (count = 0) but isn't the smallest, it means our assumption of forming consecutive groups from the smallest available card has been violated

                if card_count[current_card + i] == 0:
                    if current_card + i != heapq.heappop(min_heap):
                        return False

        return True

### Solution 2, Most Optimum, Frequency Map + Greedy with Sorting
Solution description
- Time Complexity: O(n log n)
    - Counter(hand): O(n)
    - sorted(card_count): O(k log k) where k = unique cards ≤ n, worst case O(n log n)
    Main loop: Each card is processed exactly once when forming groups. Total of n/groupSize groups × - groupSize operations = O(n)
    - Overall: O(n log n) (sorting dominates)

- Space Complexity: O(N)
    - card_count: O(k) where k = unique cards, worst case O(n)
    - sorted(card_count): O(k) for the sorted list, worst case O(n)
    - Overall: O(n) in worst case, or more precisely O(k) where k is the number of unique card values


In [ ]:
class Solution:
    def isNStraightHand(self, hand: List[int], groupSize: int) -> bool:
        if len(hand) % groupSize != 0:
            return False
        
        card_count = Counter(hand)
        
        # Process cards in sorted order
        for card in sorted(card_count):
            # While there are still instances of this card left
            while card_count[card] > 0:
                # Try to form a group starting at 'card'
                for next_card in range(card, card + groupSize):
                    if card_count[next_card] == 0:
                        return False
                    card_count[next_card] -= 1
        
        return True